In [1]:
## ds
import numpy as np
import pandas as pd
from sklearn import datasets
## data (pre)processing
from sklearn.model_selection import train_test_split
import optuna
from optuna.samplers import TPESampler
## models
import xgboost as xgb
## loss metrics
from sklearn.metrics import f1_score, confusion_matrix, classification_report
## ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Import data

In [2]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# Train model

In [3]:
def objective(trial):
    d_train = xgb.DMatrix(X_train, label=y_train)
    d_test = xgb.DMatrix(X_test, label=y_test)

    param_grid = {
        'objective':'multi:softmax',
        'num_class': 3,
        'max_depth': trial.suggest_int('max_depth', 4, 20),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, 100),
        'eta': 0.008,
        'subsample': trial.suggest_discrete_uniform('subsample', 0.1, 0.9, 0.1),
        'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.1, 0.9, 0.1),
        'reg_lambda': trial.suggest_int('reg_lambda', 1, 50),
        'reg_alpha': trial.suggest_int('reg_alpha', 0, 50),
        'verbosity': 0
    } 

    bst = xgb.train(param_grid, d_train)
    y_pred = np.rint(bst.predict(d_test))
    f1 = f1_score(y_test, y_pred, average='macro')
    return f1

In [4]:
train_time = 1 * 10 * 20 # h * m * s
study = optuna.create_study(direction='maximize', study_name='XGBClassifier')
study.optimize(objective, timeout=train_time, n_trials=1000)

print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial

print(f'\tValue: {trial.value}')
print('\tParams: ')
for key, value in trial.params.items():
    print(f'\t\t{key}: {value}')

[I 2021-08-13 01:19:43,290] A new study created in memory with name: XGBClassifier
[I 2021-08-13 01:19:43,361] Trial 0 finished with value: 0.15999999999999998 and parameters: {'max_depth': 5, 'n_estimators': 700, 'subsample': 0.4, 'colsample_bytree': 0.1, 'reg_lambda': 22, 'reg_alpha': 46}. Best is trial 0 with value: 0.15999999999999998.
[I 2021-08-13 01:19:43,374] Trial 1 finished with value: 0.9229629629629629 and parameters: {'max_depth': 17, 'n_estimators': 1000, 'subsample': 0.8, 'colsample_bytree': 0.1, 'reg_lambda': 45, 'reg_alpha': 0}. Best is trial 1 with value: 0.9229629629629629.
[I 2021-08-13 01:19:43,398] Trial 2 finished with value: 0.5239085239085239 and parameters: {'max_depth': 20, 'n_estimators': 500, 'subsample': 0.7000000000000001, 'colsample_bytree': 0.1, 'reg_lambda': 14, 'reg_alpha': 20}. Best is trial 1 with value: 0.9229629629629629.
[I 2021-08-13 01:19:43,409] Trial 3 finished with value: 0.15999999999999998 and parameters: {'max_depth': 7, 'n_estimators': 4

Number of finished trials:  1000
Best trial:
	Value: 0.974320987654321
	Params: 
		max_depth: 8
		n_estimators: 300
		subsample: 0.6
		colsample_bytree: 0.8
		reg_lambda: 19
		reg_alpha: 12


In [5]:
xgb_params = trial.params
model = xgb.XGBClassifier(**xgb_params)
model.fit(X_train, y_train, verbose=0)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=8,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=300, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=12,
              reg_lambda=19, scale_pos_weight=None, subsample=0.6,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [6]:
y_pred = model.predict(X_test)

# Result

In [7]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(f1_score(y_test, y_pred, average='macro'))

[[12  0  0]
 [ 0 12  1]
 [ 0  0 13]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      0.92      0.96        13
           2       0.93      1.00      0.96        13

    accuracy                           0.97        38
   macro avg       0.98      0.97      0.97        38
weighted avg       0.98      0.97      0.97        38

0.974320987654321
